In [1]:
import sys
import flap

sys.path.append(r"C:\marci main\Fusion\nti-wavelet-tools\python\utility")
import convert_dict_to_flap
sys.path.append(r"C:\marci main\Fusion\nti-wavelet-tools\python\core")
import core

INIT flap storage


In [4]:
path = r"C:\marci main\fusion\inputdata\AUGD_32326_MHA.sav"
print(path)

data = core.NWTDataObject()
data.load_raw_sav(path)

C:\marci main\fusion\inputdata\AUGD_32326_MHA.sav


In [7]:
print(data.raw_data.data)
flap.list_data_objects(data.raw_data)

[[-1.2256123  -1.2666433  -1.061512   ...  0.68207264  0.6513053
   0.6307938 ]
 [-0.90766746 -0.8974117  -0.7743345  ...  0.43591818  0.57950693
   0.50771654]
 [-1.061512   -1.2153565  -0.8461329  ...  0.73335946  0.77438253
   0.7436152 ]
 ...
 [ 0.0506649   0.0311327   0.0152626  ... -0.14343834 -0.15320492
  -0.15686703]
 [ 0.06531429  0.05310631  0.06287289 ... -0.04699707 -0.04699707
  -0.05432224]
 [ 0.2984829   0.2874956   0.2874956  ...  0.09949589  0.09827518
   0.08728886]]

-----------------------------
<1>(data_source:"" exp_id:"AUGD-32326") data_title:"raw data" shape:[24,200001][no error]
  Data name:"unit", unit:"a.u."
  Coords:
    'Time [s](Dims:1, Shape:[200001]) [<R. symm.>] Val. range:  8.500E-01 -  9.500E-01
    'Channels [a.u.](Dims:0, Shape:[24]) [<R. symm.>] Val:MHA-B31-14, MHA-B31-03, MHA-B31-01, MHA-B31-02, MHA-B31-12, MHA-B31-13, MHA-C04-01, MHA-C09-01_, MHA-B31-11, MHA-B31-10, ...
    'Theta [rad](Dims:0, Shape:[24]) [<R. symm.>] Val. range:  9.548E-02 -  

'\n-----------------------------\n<1>(data_source:"" exp_id:"AUGD-32326") data_title:"raw data" shape:[24,200001][no error]\n  Data name:"unit", unit:"a.u."\n  Coords:\n    \'Time [s](Dims:1, Shape:[200001]) [<R. symm.>] Val. range:  8.500E-01 -  9.500E-01\n    \'Channels [a.u.](Dims:0, Shape:[24]) [<R. symm.>] Val:MHA-B31-14, MHA-B31-03, MHA-B31-01, MHA-B31-02, MHA-B31-12, MHA-B31-13, MHA-C04-01, MHA-C09-01_, MHA-B31-11, MHA-B31-10, ...\n    \'Theta [rad](Dims:0, Shape:[24]) [<R. symm.>] Val. range:  9.548E-02 -  6.181E+00\n    \'Phi [rad](Dims:0, Shape:[24]) [<R. symm.>] Val. range:  0.000E+00 -  6.791E+00'

In [8]:
path2 = r"C:\marci main\fusion\inputdata\proba"
data.save(path2)

In [16]:
data2 = core.NWTDataObject()
data2.load(path2)
data2.save(r"C:\marci main\fusion\inputdata\proba2")
flap.list_data_objects(data2.raw_data)
for i in data2:
    print(i)

TypeError: 'NWTDataObject' object is not iterable